In [ ]:
# def syn_res(syn_string, time_ind, spike_timing_mat, h, type_syn=None):
def syn_res(syn_string,type_syn,t,time,i,j,w_ij,del_i,h,M):  # spike in neuron i, produces a synaptic current in neuron j, weight = w_ij
    syn_curr = np.zeros((M,))

    ts_ds = np.float64(time[t]) + del_i
    ind = int(np.where(time == ts_ds))
    if syn_string == "static":
        syn_curr[ind] = w_ij*1/h
        # result_mat = [1 if time_ind == a_spike else 0 for a_spike in spike_timing_mat]
        # return result_mat
    elif syn_string == "first-order":
        tau_s = 4 * h # Given in paper
        syn_curr[ind:M] = w_ij * (1/tau_s) * np.exp(-(1/tau_s)*(time[ind:M] -ts_ds))
        # result_mat = (1/tau_s) * (np.exp(- (time_ind - spike_timing_mat)/tau_s) 
        # * np.heaviside(time_ind - spike_timing_mat))
        # return result_mat
    elif syn_string == "second-order":
        # if type_syn is None:
            # raise ValueError('type required')
        elif type_syn == 1:
            tau_s1, tau_s2 = 4, 8
        elif type_syn == 0:
            tau_s1, tau_s2 = 4, 2
        # else:
            # raise ValueError('type invalid')
        
        syn_curr[ind:M] = (w_ij/(tau_s1-tau_s2)) * (np.exp(-(1/tau_s1)*(time[ind:M] -ts_ds)) -np.exp(-(1/tau_s2)*(time[ind:M] -ts_ds)))
        # result_mat = ((1/(tau_s1 - tau_s2)) * np.exp(- (time_ind - spike_timing_mat)/tau_s1)
        #               * np.heaviside(time_ind - spike_timing_mat)
        #               - ((1/(tau_s1 - tau_s2)) * np.exp(- (time_ind - spike_timing_mat)/tau_s2)
        #               * np.heaviside(time_ind - spike_timing_mat)))
        # return result_mat
            
    return syn_curr

In [ ]:
def LIF_rsp_one(neuron_id,Weight_mat, Vm_prev, FanIn_list, syn_string, 
                tau_m, spikeTime_mat, Delay_mat, it_prev, h, time_ind, **kwargs):
    global vrest, vth
    spike_happened = 0
    V_temp = Vm_prev  - Vm_prev/tau_m + it_prev
    for i in FanIn_list:
        V_temp += Weight_mat[i] * syn_res(syn_string, time_ind, spikeTime_mat[:, i] + Delay_mat[i], h) 
    if time_ind - spikeTime_mat[neuron_id][-1] <= tf.math.ceil(t_refrac/h):
        V_temp = vrest
    elif V_temp > vth:
        spikeTime_mat[time_ind, neuron_id] = 1
        V_temp = vrest
    else:
        pass
    return V_temp
    
    